In [1]:
import ee
import geemap
import os
import shapely
import json
import mgrs
import pandas as pd
import ipywidgets
import folium
import geemap.eefolium as gmap
import traceback

# Initialize Map Object

In [2]:
ee.Initialize()

In [3]:
Map = gmap.Map()

# Import Polygons

In [5]:
def import_aois(csv_loc):    

        df_labels = pd.read_csv(csv_loc)
        df_labels = df_labels[["center-lat","center-long","polygon","Labels combined"]]

        polygons = []
        for polygon in df_labels["polygon"]:
            polygons.append(json.loads(polygon)["coordinates"])


        tiles = []
        tiles_dic = {}
        polygon_id = 0 
        coordinates = []
        m = mgrs.MGRS()
        for items in polygons:
            polygon_id += 1 
            for item in items:
                for lon_lat in item:
                    coordinates.append(lon_lat)
                    c = m.toMGRS(lon_lat[1], lon_lat[0])
                    tile = c[0:5]
                    

                    if polygon_id in tiles_dic:

                        tiles_dic[polygon_id].append(tile)

                    else:

                        tiles_dic[polygon_id] = [tile]

                    tiles.append(tile)

                tiles_dic[polygon_id] = list(set(tiles_dic[polygon_id]))

        tiles = list(set(tiles))

        df_labels["tiles"] = tiles_dic.values()

        #bounding box

        min_lon = min([i[0] for i in coordinates])
        min_lat = min([i[1] for i in coordinates])
        max_lon = max([i[0] for i in coordinates])
        max_lat = max([i[1] for i in coordinates])

        bounding_box = min_lon,min_lat,max_lon,max_lat
        
        return df_labels,polygons

In [6]:
df_labels, polygons = import_aois("/Volumes/Lacie/zhenyadata/Project_Canopy_Data/PC_Data/Sentinel_Data/Labelled/Tiles_v3/Polygon_List/polygons_101320.csv")

/Users/purgatorid/opt/anaconda3/envs/gee-conda/lib/python3.8/site-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


In [7]:
polygons[2]

[[[13.148403, 5.597193],
  [13.199902, 5.622648],
  [13.227882, 5.608811],
  [13.227539, 5.568491],
  [13.148403, 5.597193]]]

In [7]:
# # region of interest
# geom = ee.Geometry.Polygon(polygons[0])



In [8]:
# help(ee.ImageCollection)

In [9]:
# # image collection
# ic = ee.ImageCollection('COPERNICUS/S2')\
#     .filterBounds(geom)\
#     .filterDate(startDate,stopDate)
    
# # # single image
# # img = ee.Image(ic.first())

In [10]:
# ee.Image(ic.get(0)).getInfo()

In [25]:
help(Map.add_layer)

Help on method add_layer in module geemap.eefolium:

add_layer(ee_object, vis_params={}, name='Layer untitled', shown=True, opacity=1.0) method of geemap.eefolium.Map instance
    Adds a given EE object to the map as a layer.
    
    Args:
        ee_object (Collection|Feature|Image|MapId): The object to add to the map.
        vis_params (dict, optional): The visualization parameters. Defaults to {}.
        name (str, optional): The name of the layer. Defaults to 'Layer untitled'.
        shown (bool, optional): A flag indicating whether the layer should be on by default. Defaults to True.
        opacity (float, optional): The layer's opacity represented as a number between 0 and 1. Defaults to 1.



In [38]:
Map = gmap.Map()

# start and end of time series
startDate = ee.Date('2020-01-01')
stopDate  = ee.Date('2020-12-31')
TCI_RGB = ['TCI_R', 'TCI_G', 'TCI_B']
# RGB = ['B11', 'B8', 'B3']
# G = ['B1']
count_loop = 0
count_bad = 0  
good_images = []
good_dict = {}
feature_id = 0 
features = []


for poly in polygons[0:3]:
    Map = gmap.Map()
    # create an roi. first item in Misha's label list
    feature_id += 1 
    polys = ee.Geometry.Polygon(poly)
    feature = ee.Feature(polys,{"name":feature_id})
    features.append(feature)
    


    # Add Earth Engine dataset
    # Sentinel 2 ImageCollection
    # filter by bounds and data
    collection = ee.ImageCollection('COPERNICUS/S2_SR')\
        .filterBounds(polys)\
        .filterDate(startDate,stopDate)\
        .filter(ee.Filter.lt("CLOUDY_PIXEL_PERCENTAGE",5))\
        .sort('CLOUD_COVER',True)\
        .select(TCI_RGB)

    # create a FeatureCollection based on the roi and center the map
#     centroid = polys.centroid()
#     lng, lat = centroid.getInfo()['coordinates']
#     print("lng = {}, lat = {}".format(lng, lat))
    # Map.setCenter(lng, lat, 12)
    fc = ee.FeatureCollection(polys)

    # filter the ImageCollection using the roi
    # collection_filtered = collection.filterBounds(polys)
    # mosaic = collection_filtered.mosaic()

    # use only first image from collection
#     img = ee.Image(collection.first())
#     img = img.clip(polys)

#     # use the mean image
#     collection_mean = collection.mean()


#     # print out the number of images in the ImageCollection
#     count = collection_filtered.size().getInfo()
#     print("Count: ", count)

#     # add the ImageCollection and the roi to the map
#     vis = {'bands': TCI_RGB, min: 0 , max: 3200}
#     Map.addLayer(fc, {"color": 'FF0000'},feature_id)
#     Map.add_layer(fc, {}, 'default display')
#     Map.addLayer(mosaic,vis)
#     # vis = {min: 0 , max : 3200}
#     # Map.addLayer(mosaic,vis)
#     Map.addLayer(mosaic)
#     Map.addLayer(img,vis)
#     # Map.addLayer(collection_mean,vis)
#     # Map.addLayer(collection,vis,"Original Collection")
#     # Map.addLayer(collection_mean,vis, "Collection Mean")



#     try:
#         Map.addLayer(img,vis, f"Image ID {count_loop}")
#         count_loop += 1
#         good_dict[count_loop] = count_loop - 1 
#         good_images.append(img)
#         print(f"Added Image ID {count_loop}")
        
    
#     except Exception:            
#         print(f"Image ID {count_loop}")
#         count_loop += 1
#         good_dict[count_loop] = None
#         traceback.print_exc()
#         count_bad += 1 

    

# print(f'found {count_bad} polygons without a corresponding image collection')


# #Create custom collection with list of images for DL
# custom_collection = ee.ImageCollection.fromImages(good_images)
# image_dl_list = custom_collection.toList(105)

# add all polygon features to map

fc = ee.FeatureCollection(features)
Map.add_layer(fc, {}, 'default display')
Map

In [37]:
ee.FeatureCollection(features[0])

EEException: Invalid GeoJSON geometry.

In [29]:
Map

# Rename Export Files to their corresponding ID from spreadsheet 

In [151]:
import os
from glob import glob 

# os.raneme

tif_loc = "/Volumes/Lacie/zhenyadata/Project_Canopy_Data/PC_Data/Sentinel_Data/GEE/tiles_v3/polygon_exports/test/v1/*.tif" 
list_tif = glob(tif_loc)

        
count_good = 0 
total_count_offset = 0 
start = 101
index = -1

while start > 0: 
    if isinstance(good_dict[start],int):
        base_path = ("/".join(list_tif[index].split("/")[:-1]) + "/")
        # print(base_path)
        # print("from filename:",list_tif[index],"to filename:",start)
        print("from filename:",list_tif[index],"to filename:",base_path + str(start) + ".tif")
        os.rename(list_tif[index],base_path + str(start) + ".tif")
        index -= 1
        start -= 1
    else:
        start -= 1


        



from filename: /Volumes/Lacie/zhenyadata/Project_Canopy_Data/PC_Data/Sentinel_Data/GEE/tiles_v3/polygon_exports/test/v1/97.tif to filename: /Volumes/Lacie/zhenyadata/Project_Canopy_Data/PC_Data/Sentinel_Data/GEE/tiles_v3/polygon_exports/test/v1/101.tif
from filename: /Volumes/Lacie/zhenyadata/Project_Canopy_Data/PC_Data/Sentinel_Data/GEE/tiles_v3/polygon_exports/test/v1/96.tif to filename: /Volumes/Lacie/zhenyadata/Project_Canopy_Data/PC_Data/Sentinel_Data/GEE/tiles_v3/polygon_exports/test/v1/100.tif
from filename: /Volumes/Lacie/zhenyadata/Project_Canopy_Data/PC_Data/Sentinel_Data/GEE/tiles_v3/polygon_exports/test/v1/95.tif to filename: /Volumes/Lacie/zhenyadata/Project_Canopy_Data/PC_Data/Sentinel_Data/GEE/tiles_v3/polygon_exports/test/v1/99.tif
from filename: /Volumes/Lacie/zhenyadata/Project_Canopy_Data/PC_Data/Sentinel_Data/GEE/tiles_v3/polygon_exports/test/v1/94.tif to filename: /Volumes/Lacie/zhenyadata/Project_Canopy_Data/PC_Data/Sentinel_Data/GEE/tiles_v3/polygon_exports/test/

In [ ]:
start > 0 

In [9]:
image_dl_list.get(1).getInfo()["id"]

'COPERNICUS/S2_SR/20200105T094309_20200105T095703_T32NMM'

In [16]:
RGB = ['TCI_R', 'TCI_G', 'TCI_B']

# Download the specific image
# for image_obj in range(len(good_images)):
for i in range(len(good_images)):
    downConfig = {'scale': 10, "maxPixels": 1.0E13, 'driveFolder': 'image3',"driveFileNamePrefix":str(i)}  # scale means resolution.
    image_to_dl = ee.Image(image_dl_list.get(i))
    # img_2 = image_to_dl.select('B.+')
    img_2 = image_to_dl.select(RGB)
    name = img_2.getInfo()["id"].split("/")[-1]
    print("Image to Download:",name)
    task = ee.batch.Export.image(img_2, name, downConfig)
    task.start()

while task.status()["state"] == 'RUNNING':
    print(task.status(), end="\r", flush=True)

Image to Download: 20200105T094309_20200105T095703_T32NNM
Image to Download: 20200105T094309_20200105T095703_T32NMM
Image to Download: 20200104T092401_20200104T093437_T33NTG
Image to Download: 20200104T092401_20200104T093437_T33NVF
Image to Download: 20200104T092401_20200104T093437_T33NTF
Image to Download: 20200105T094309_20200105T095703_T32NML
Image to Download: 20200104T092401_20200104T093437_T33NUF
Image to Download: 20200104T092401_20200104T093437_T33NVF
Image to Download: 20200107T093351_20200107T094442_T32NRL
Image to Download: 20200107T093351_20200107T094442_T32NRL
Image to Download: 20200114T092331_20200114T093418_T33NVE
Image to Download: 20200105T094309_20200105T095703_T32NML
Image to Download: 20200114T092331_20200114T093418_T33NVE
Image to Download: 20200213T092101_20200213T093321_T32NRK
Image to Download: 20200205T091049_20200205T091719_T33NWE
Image to Download: 20200205T091049_20200205T091719_T33NWE
Image to Download: 20200205T091049_20200205T091719_T33NXE
Image to Downl

In [46]:
image_to_dl.getInfo()

{'type': 'Image',
 'bands': [{'id': 'TCI_R',
   'data_type': {'type': 'PixelType',
    'precision': 'float',
    'min': 0,
    'max': 0.02550000138580799},
   'dimensions': [1767, 1580],
   'origin': [984, 5384],
   'crs': 'EPSG:32632',
   'crs_transform': [10, 0, 499980, 0, -10, 700020]},
  {'id': 'TCI_G',
   'data_type': {'type': 'PixelType',
    'precision': 'float',
    'min': 0,
    'max': 0.02550000138580799},
   'dimensions': [1767, 1580],
   'origin': [984, 5384],
   'crs': 'EPSG:32632',
   'crs_transform': [10, 0, 499980, 0, -10, 700020]},
  {'id': 'TCI_B',
   'data_type': {'type': 'PixelType',
    'precision': 'float',
    'min': 0,
    'max': 0.02550000138580799},
   'dimensions': [1767, 1580],
   'origin': [984, 5384],
   'crs': 'EPSG:32632',
   'crs_transform': [10, 0, 499980, 0, -10, 700020]}],
 'properties': {'system:footprint': {'type': 'Polygon',
   'coordinates': [[[9.088783, 5.753151999999999],
     [9.179248999999999, 5.703106],
     [9.248257000000002, 5.792946],
 

In [41]:
Map = geemap.Map()
# Assemble a collection of Sentinel-2 surface reflectance images for a given region and date range.
s2col = ee.ImageCollection('COPERNICUS/S2_SR') \
  .filterBounds(ee.Geometry.Point(-96.9037, 48.0395)) \
  .filterDate('2019-06-01', '2020-10-01')

# Define visualization arguments.
visArgs = {'bands': ['B4', 'B3', 'B2'], min: 300, max: 3500}


def visFun(img):
    # return Map.addLayer(img,visArgs).copyProperties(img, img.propertyNames())
    # return Map.addLayer(img,visArgs)
    return Map.addLayer(img)

# Map over the image collection to convert each image to an RGB visualization using the previously defined visualization function.
s2colVis = visFun(s2col)

Map

Map(center=[40, -100], controls=(WidgetControl(options=['position'], widget=HBox(children=(ToggleButton(value=…

In [65]:
print(polygons[1])

# start and end of time series
startDate = ee.Date('2019-01-01')
stopDate  = ee.Date('2020-12-31')

# create an roi. first item in Misha's label list
polys = ee.Geometry.Polygon(polygons[0])

# RGB = ['B4', 'B3', 'B2']
RGB = ['TCI_R', 'TCI_G', 'TCI_B']

# Add Earth Engine dataset
# Sentinel 2 ImageCollection
# filter by bounds and data
collection = ee.ImageCollection('COPERNICUS/S2_SR')\
    .filterBounds(polys)\
    .filterDate(startDate,stopDate)\
    .filter(ee.Filter.lt("CLOUDY_PIXEL_PERCENTAGE",1))\
    .sort('CLOUD_COVER',True)
    # .select(RGB)
# # create a FeatureCollection based on the roi and center the map
centroid = polys.centroid()
lng, lat = centroid.getInfo()['coordinates']
print("lng = {}, lat = {}".format(lng, lat))
Map.setCenter(lng, lat, 12)
fc = ee.FeatureCollection(polys)

# filter the ImageCollection using the roi
mosaic = collection.mosaic()

# # # print out the number of images in the ImageCollection
count = collection.size().getInfo()
print("Count: ", count)

# add the ImageCollection and the roi to the map
# vis = {'bands': ["B4", "B3", "B2"], min: 0 , max: 3000}
vis = {min: 0 , max: 3000}
# Map.addLayer(mosaic,vis)
# Map.addLayer(collection,vis)
# Map.addLayer(fc)

# export the ImageCollection to Google Drive
downConfig = {'scale': 10, "maxPixels": 1.0E13, 'driveFolder': 'image2'}  # scale means resolution.
img_lst = collection.toList(100)

for i in range(0, 1):
    image = ee.Image(img_lst.get(i))
    # print(image)
    # print(image.getInfo())
    image = image.select('B.+')
    name = image.get('system:index').getInfo()
    print(name)
#     Map.addLayer(image,vis)
    # task = ee.batch.Export.image(image, name, downConfig)
    # task.start()
#     print(task.status())

[[[8.891888, 5.804559], [8.931885, 5.803022], [8.975143, 5.755202], [8.955746, 5.712159], [8.907509, 5.705498], [8.871632, 5.724457], [8.891888, 5.804559]]]
lng = 9.163954053102769, lat = 5.7876496002257225
Count:  4
20190110T094359_20190110T095700_T32NNM


In [64]:
img_lst.get(1).getInfo()

{'type': 'Image',
 'bands': [{'id': 'B1',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min': 0,
    'max': 65535},
   'dimensions': [1830, 1830],
   'crs': 'EPSG:32632',
   'crs_transform': [60, 0, 499980, 0, -60, 700020]},
  {'id': 'B2',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min': 0,
    'max': 65535},
   'dimensions': [10980, 10980],
   'crs': 'EPSG:32632',
   'crs_transform': [10, 0, 499980, 0, -10, 700020]},
  {'id': 'B3',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min': 0,
    'max': 65535},
   'dimensions': [10980, 10980],
   'crs': 'EPSG:32632',
   'crs_transform': [10, 0, 499980, 0, -10, 700020]},
  {'id': 'B4',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min': 0,
    'max': 65535},
   'dimensions': [10980, 10980],
   'crs': 'EPSG:32632',
   'crs_transform': [10, 0, 499980, 0, -10, 700020]},
  {'id': 'B5',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min': 0,


In [119]:
help(task)

Help on Task in module ee.batch object:

class Task(builtins.object)
 |  Task(task_id, task_type, state, config=None, name=None)
 |  
 |  A batch task that can be run on the EE batch processing system.
 |  
 |  Methods defined here:
 |  
 |  __init__(self, task_id, task_type, state, config=None, name=None)
 |      Creates a Task with the given ID and configuration.
 |      
 |      The constructor is not for public use. Instances can be obtained by:
 |      - Calling the static method Task.list().
 |      - Calling any of the methods on the Export static class.
 |      - Unpickling a previously pickled Task object.
 |      
 |      If you're looking for a task's status but don't need a full task object,
 |      ee.data.getTaskStatus() may be appropriate.
 |      
 |      Args:
 |        task_id: The task ID, originally obtained through ee.data.newTaskId().
 |          May be None if the ID is not yet known.
 |        task_type: The type of the task; one of the values in Task.Type.
 |  

In [89]:
help(image.get)

Help on method Element.get in Element:

Element.get(*args, **kwargs) method of ee.image.Image instance
    Extract a property from a feature.
    
    Args:
      object: The feature to extract the property from.
      property: The property to extract.



In [97]:
# Add Earth Engine dataset
# USDA NAIP ImageCollection
collection = ee.ImageCollection('COPERNICUS/S2')

# create an roi
polys = ee.Geometry.Polygon(
        [[[-99.29615020751953, 46.725459351792374],
          [-99.2116928100586, 46.72404725733022],
          [-99.21443939208984, 46.772037733479884],
          [-99.30267333984375, 46.77321343419932]]])

# create a FeatureCollection based on the roi and center the map
centroid = polys.centroid()
lng, lat = centroid.getInfo()['coordinates']
print("lng = {}, lat = {}".format(lng, lat))
Map.setCenter(lng, lat, 12)
fc = ee.FeatureCollection(polys)

# filter the ImageCollection using the roi
naip = collection.filterBounds(polys)
naip_2015 = naip.filterDate('2020-01-01', '2020-12-31')
mosaic = naip_2015.mosaic()

# print out the number of images in the ImageCollection
count = naip_2015.size().getInfo()
print("Count: ", count)

# add the ImageCollection and the roi to the map
vis = {'bands': ["B4", "B3", "B2"]}
Map.addLayer(mosaic,vis)
Map.addLayer(fc)

# export the ImageCollection to Google Drive
downConfig = {'scale': 30, "maxPixels": 1.0E13, 'driveFolder': 'image'}  # scale means resolution.
img_lst = naip_2015.toList(100)

for i in range(0, count):
    image = ee.Image(img_lst.get(i))
    # name = image.get('system:time_start').getInfo()
    # name = image.get('CLOUDY_PIXEL_PERCENTAGE').getInfo()
    name = image.get('system:index').getInfo()
    print(name)
    task = ee.batch.Export.image(image, name, downConfig)
    task.start()

lng = -99.25621066337796, lat = 46.748869171015414
Count:  121
20200101T173721_20200101T173724_T14TMS


KeyboardInterrupt: 

In [19]:
help(image.getInfo()))

TypeError: getInfo() takes 1 positional argument but 2 were given